In [49]:
using DataFrames, MLJ, Rasters, Plots

In [5]:
# Read raster
s2 = Raster("../../sentinel2_subset.tif")

1748×1124×13 Raster{UInt16,3} with dimensions: 
  X Projected range(443690.0, stop=461160.0, length=1748) ForwardOrdered Regular Intervals crs: WellKnownText,
  Y Projected range(5.38158e6, stop=5.37035e6, length=1124) ReverseOrdered Regular Intervals crs: WellKnownText,
  Band Categorical 1:13 ForwardOrdered

from file:
../../sentinel2_subset.tif

In [56]:
# Subset
img = s2[:,:,[2,3,4,8]]
nrows = size(img)[1]
ncols = size(img)[2]
nbands = size(img)[3]

4

In [57]:
# Create a dataframe
df = img |>
□ -> reshape(Array(□), (size(□)[1]*size(□)[2], size(□)[3])) |>
□ -> DataFrame(□, :auto)

,x1,x2,x3,x4
,UInt16,UInt16,UInt16,UInt16
1,881,658,527,2203
2,835,675,429,2429
3,806,599,349,2033
4,806,619,327,1985
5,824,652,355,2142
6,853,643,365,2031
7,815,618,345,1815
8,814,579,332,1420
9,795,550,308,1372


In [51]:
# K-Means pipeline
model_kmeans = @load KMeans pkg=Clustering
pipe_kmeans = @pipeline Standardizer(count=true) model_kmeans(k=10)

import MLJClusteringInterfaceimport MLJClusteringInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\julio\.julia\packages\MLJModels\Ci1zC\src\loading.jl:168
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\julio\.julia\packages\MLJModels\Ci1zC\src\loading.jl:168


Pipeline295(
    standardizer = Standardizer(
            features = Symbol[],
            ignore = false,
            ordered_factor = false,
            count = true),
    k_means = KMeans(
            k = 10,
            metric = Distances.SqEuclidean(0.0)))

In [53]:
# Fit the model and get the report
machine_kmeans = machine(pipe_kmeans, df) |> fit!
report_kmeans = report(machine_kmeans)

┌ Info: Training Machine{Pipeline295,…}.
└ @ MLJBase C:\Users\julio\.julia\packages\MLJBase\pCiRR\src\machines.jl:464
┌ Info: Training Machine{Standardizer,…}.
└ @ MLJBase C:\Users\julio\.julia\packages\MLJBase\pCiRR\src\machines.jl:464
┌ Info: Training Machine{KMeans,…}.
└ @ MLJBase C:\Users\julio\.julia\packages\MLJBase\pCiRR\src\machines.jl:464


(k_means = (assignments = [1, 4, 6, 6, 4, 4, 6, 10, 10, 6  …  8, 8, 8, 8, 8, 7, 8, 8, 8, 7],
            cluster_labels = CategoricalArrays.CategoricalValue{Int64, UInt32}[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],),
 standardizer = (features_fit = [:x2, :x3, :x4, :x1],),
 machines = Machine[Machine{Standardizer,…}, Machine{KMeans,…}],
 report_given_machine = OrderedCollections.LittleDict{Any, Any, Vector{Any}, Vector{Any}}(Machine{Standardizer,…} => (features_fit = [:x2, :x3, :x4, :x1],), Machine{KMeans,…} => (assignments = [1, 4, 6, 6, 4, 4, 6, 10, 10, 6  …  8, 8, 8, 8, 8, 7, 8, 8, 8, 7], cluster_labels = CategoricalArrays.CategoricalValue{Int64, UInt32}[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])),)

In [55]:
# Add a new column with the clusters
clusters = collect(report_kmeans.k_means.assignments);

In [59]:
typeof(clusters)

Vector{Int64} (alias for Array{Int64, 1})

In [60]:
# Create the output raster
kmeans_raster = reshape(clusters, (nrows, ncols, 1)) 
# |>
# □ -> reverse(□, dims=2) |>
# □ -> Raster(□)
#  |>
# □ -> GeoArrays.bbox!(□, boundary_box) |>
# □ -> GeoArrays.epsg!(□, 32610)

1748×1124×1 Array{Int64, 3}:
[:, :, 1] =
  1   1   7   1   6  10  10  10  10  10  …   1   8  8  7  7  1  6  4  4  4  4
  4   1   1   6   6  10  10  10   6  10      1   4  8  8  8  6  4  4  4  4  4
  6   6   6  10  10  10  10   6   4   4      8   8  8  4  8  4  4  8  4  8  4
  6   6   6   6  10  10   6   4   4   4      8   8  1  1  8  4  4  8  8  8  6
  4   6   6  10  10   6   6   6   4   6      8   8  1  6  4  4  8  8  8  8  6
  4   6  10  10  10   6   6   6   6   4  …   8   8  4  4  8  8  8  8  8  8  4
  6   6   6   6   6   6   4   4   6   4      1   1  4  8  8  8  8  8  8  8  6
 10   6   6   6   6   6   6   4   4   6      1   8  8  8  8  8  8  8  8  8  6
 10  10  10   6   6   6   6   4   4   6      7   8  4  8  8  8  4  8  8  7  1
  6  10   6   1   1   1   6   6   4   4      7   7  1  8  8  8  8  8  8  8  4
  ⋮                   ⋮                  ⋱          ⋮              ⋮        
  6   6   6   6  10  10   6   6   6   6     10  10  6  6  4  8  7  7  1  1  8
  6   4   6   6  10  10 